# Neural Networks

In this task you are suppose to implement 2 types of mulilayer Perceptrons: 

1. Using only Python. 
2. Using a high level library

Download the Ecoli dataset: https://archive.ics.uci.edu/ml/datasets/Ecoli
* Predict the two classes: cp and im (remove the rest of the dataset).
* Make the necessary adjustments to the data.
* Implement and test a Multilayer Perceptron from scratch using only Python and standard libraries (do not train this).
* Implement, train and test a Multilayer Perceptron using a high level library (e.g., Torch, Keras, TensorFlow).
* Choose the network architecture with care.
* Train and validate all algorithms.
* Make the necessary assumptions.

# Load the data

In [57]:
import pandas as pd

df = pd.read_csv('ecoli/ecoli.data', header=None, delim_whitespace=True)

# Remove all rows not containing everything except cp and im and drop the non-numeric columns but keep label
df = df[df[8].isin(['cp', 'im'])].drop([0], axis=1)
print(df)

        1     2     3    4     5     6     7   8
0    0.49  0.29  0.48  0.5  0.56  0.24  0.35  cp
1    0.07  0.40  0.48  0.5  0.54  0.35  0.44  cp
2    0.56  0.40  0.48  0.5  0.49  0.37  0.46  cp
3    0.59  0.49  0.48  0.5  0.52  0.45  0.36  cp
4    0.23  0.32  0.48  0.5  0.55  0.25  0.35  cp
..    ...   ...   ...  ...   ...   ...   ...  ..
215  0.10  0.49  0.48  0.5  0.41  0.67  0.21  im
216  0.30  0.51  0.48  0.5  0.42  0.61  0.34  im
217  0.61  0.47  0.48  0.5  0.00  0.80  0.32  im
218  0.63  0.75  0.48  0.5  0.64  0.73  0.66  im
219  0.71  0.52  0.48  0.5  0.64  1.00  0.99  im

[220 rows x 8 columns]


# Shuffle the data and split into training and test data

In [58]:
# Shuffle the data
df = df.sample(frac=1).reset_index(drop=True)
y = df.iloc[:, -1]
X = df.iloc[:, :-1]

# Split the data into training and test data 80/20
X_train = X.iloc[:int(len(X)*0.8)]
X_test = X.iloc[int(len(X)*0.8):]
y_train = y.iloc[:int(len(y)*0.8)]
y_test = y.iloc[int(len(y)*0.8):]

# Use sklearn to create an MLP Classifier

In [59]:
from sklearn.model_selection import GridSearchCV
import sklearn.neural_network as nn
from sklearn.metrics import classification_report, confusion_matrix

# Use cross validation to find the best parameters
param_grid = {'hidden_layer_sizes': [(2, 3), (3, 3), (3, 4), (4, 4), (4, 5), (5, 5), (5, 6), (6, 6), (6, 7),
                                      (7, 7), (7, 8), (8, 8), (8, 9), (9, 9), (9, 10), (10, 10)],
               'activation': ['identity', 'tanh', 'relu'],
               'solver': ['lbfgs', 'sgd'],
               'max_iter': [100000]}
grid = GridSearchCV(nn.MLPClassifier(), param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)

{'activation': 'relu', 'hidden_layer_sizes': (4, 5), 'max_iter': 100000, 'solver': 'lbfgs'}


In [60]:
# Evaluate the model
y_pred = grid.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[27  1]
 [ 0 16]]
              precision    recall  f1-score   support

          cp       1.00      0.96      0.98        28
          im       0.94      1.00      0.97        16

    accuracy                           0.98        44
   macro avg       0.97      0.98      0.98        44
weighted avg       0.98      0.98      0.98        44
